In [1]:

import torch
from torch import nn, optim
from torch.nn import functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import MinMaxScaler
import time
import math

In [2]:
# Check for CUDA availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [3]:
# Load and preprocess the data
df = pd.read_csv("../data/processed/uav_hugging_face_dropped_20.csv")
print(f"Original data shape: {df.shape}")

Original data shape: (176, 4)


In [4]:
# Extract data
timestamps = df['timestamp'].values
positions = df[['tx', 'ty', 'tz']].values

In [5]:

# Calculate time deltas between consecutive points
time_deltas = np.diff(timestamps)
time_deltas = np.insert(time_deltas, 0, 0)  # Add 0 for the first point


In [6]:
# Scale coordinates to improve training
scaler = MinMaxScaler(feature_range=(-1, 1))
positions_scaled = scaler.fit_transform(positions)

In [7]:
# Create torch tensors
tensor_timestamps = torch.tensor(timestamps - timestamps[0], dtype=torch.float32).to(device)
tensor_positions = torch.tensor(positions_scaled, dtype=torch.float32).to(device)
tensor_time_deltas = torch.tensor(time_deltas, dtype=torch.float32).to(device)


In [8]:
# Define the ODE function base class (similar to what you had in test.py)
class ODEF(nn.Module):
    def forward_with_grad(self, z, t, grad_outputs):
        """Compute f and a df/dz, a df/dp, a df/dt"""
        batch_size = z.shape[0]

        out = self.forward(z, t)

        a = grad_outputs
        adfdz, adfdt, *adfdp = torch.autograd.grad(
            (out,), (z, t) + tuple(self.parameters()), grad_outputs=(a),
            allow_unused=True, retain_graph=True
        )
        # grad method automatically sums gradients for batch items, we have to expand them back 
        if adfdp and any(p is not None for p in adfdp):
            adfdp = torch.cat([p_grad.flatten() if p_grad is not None else 
                            torch.zeros_like(p).flatten() 
                            for p_grad, p in zip(adfdp, self.parameters())]).unsqueeze(0)
            adfdp = adfdp.expand(batch_size, -1) / batch_size
        else:
            adfdp = torch.zeros(batch_size, 0).to(z)
        if adfdt is not None:
            adfdt = adfdt.expand(batch_size, 1) / batch_size
        else:
            adfdt = torch.zeros(batch_size, 1).to(z)
        return out, adfdz, adfdt, adfdp

    def flatten_parameters(self):
        p_shapes = []
        flat_parameters = []
        for p in self.parameters():
            p_shapes.append(p.size())
            flat_parameters.append(p.flatten())
        return torch.cat(flat_parameters) if len(flat_parameters) > 0 else torch.tensor([]).to(device)

In [9]:
def ode_solve(z0, t0, t1, f):
    """
    Simplest Euler ODE initial value solver
    """
    h_max = 0.05
    n_steps = math.ceil((abs(t1 - t0)/h_max).max().item())

    h = (t1 - t0)/n_steps
    t = t0
    z = z0

    for i_step in range(n_steps):
        z = z + h * f(z, t)
        t = t + h
    return z

In [10]:
class ODEAdjoint(torch.autograd.Function):
    @staticmethod
    def forward(ctx, z0, t, flat_parameters, func):
        assert isinstance(func, ODEF)
        bs, *z_shape = z0.size()
        time_len = t.size(0)

        with torch.no_grad():
            z = torch.zeros(time_len, bs, *z_shape).to(z0)
            z[0] = z0
            for i_t in range(time_len - 1):
                z0 = ode_solve(z0, t[i_t], t[i_t+1], func)
                z[i_t+1] = z0

        ctx.func = func
        ctx.save_for_backward(t, z.clone(), flat_parameters)
        return z

    @staticmethod
    def backward(ctx, dLdz):
        """
        dLdz shape: time_len, batch_size, *z_shape
        """
        func = ctx.func
        t, z, flat_parameters = ctx.saved_tensors
        time_len, bs, *z_shape = z.size()
        # n_dim = np.prod(z_shape)
        n_dim = 3
        n_params = flat_parameters.size(0)

        # Dynamics of augmented system to be calculated backwards in time
        def augmented_dynamics(aug_z_i, t_i):
            """
            tensors here are temporal slices
            t_i - is tensor with size: bs, 1
            aug_z_i - is tensor with size: bs, n_dim*2 + n_params + 1
            """
            z_i, a = aug_z_i[:, :n_dim], aug_z_i[:, n_dim:2*n_dim]  # ignore parameters and time

            # Unflatten z and a
            z_i = z_i.view(bs, *z_shape)
            a = a.view(bs, *z_shape)
            with torch.set_grad_enabled(True):
                t_i = t_i.detach().requires_grad_(True)
                z_i = z_i.detach().requires_grad_(True)
                func_eval, adfdz, adfdt, adfdp = func.forward_with_grad(z_i, t_i, grad_outputs=a)  # bs, *z_shape
                adfdz = adfdz.to(z_i) if adfdz is not None else torch.zeros(bs, *z_shape).to(z_i)
                adfdp = adfdp.to(z_i) if adfdp is not None else torch.zeros(bs, n_params).to(z_i)
                adfdt = adfdt.to(z_i) if adfdt is not None else torch.zeros(bs, 1).to(z_i)

            # Flatten f and adfdz
            func_eval = func_eval.view(bs, n_dim)
            adfdz = adfdz.view(bs, n_dim) 
            return torch.cat((func_eval, -adfdz, -adfdp, -adfdt), dim=1)

        dLdz = dLdz.view(time_len, bs, n_dim)  # flatten dLdz for convenience
        with torch.no_grad():
            ## Create placeholders for output gradients
            # Prev computed backwards adjoints to be adjusted by direct gradients
            adj_z = torch.zeros(bs, n_dim).to(dLdz)
            adj_p = torch.zeros(bs, n_params).to(dLdz)
            # In contrast to z and p we need to return gradients for all times
            adj_t = torch.zeros(time_len, bs, 1).to(dLdz)

            for i_t in range(time_len-1, 0, -1):
                z_i = z[i_t]
                t_i = t[i_t]
                f_i = func(z_i, t_i).view(bs, n_dim)

                # Compute direct gradients
                dLdz_i = dLdz[i_t]
                dLdt_i = torch.bmm(torch.transpose(dLdz_i.unsqueeze(-1), 1, 2), f_i.unsqueeze(-1))[:, 0]

                # Adjusting adjoints with direct gradients
                adj_z += dLdz_i
                adj_t[i_t] = adj_t[i_t] - dLdt_i

                # Pack augmented variable
                aug_z = torch.cat((z_i.view(bs, n_dim), adj_z, torch.zeros(bs, n_params).to(z), adj_t[i_t]), dim=-1)

                # Solve augmented system backwards
                aug_ans = ode_solve(aug_z, t_i, t[i_t-1], augmented_dynamics)

                # Unpack solved backwards augmented system
                adj_z[:] = aug_ans[:, n_dim:2*n_dim]
                adj_p[:] += aug_ans[:, 2*n_dim:2*n_dim + n_params]
                adj_t[i_t-1] = aug_ans[:, 2*n_dim + n_params:]

                del aug_z, aug_ans

            ## Adjust 0 time adjoint with direct gradients
            # Compute direct gradients 
            dLdz_0 = dLdz[0]
            dLdt_0 = torch.bmm(torch.transpose(dLdz_0.unsqueeze(-1), 1, 2), f_i.unsqueeze(-1))[:, 0]

            # Adjust adjoints
            adj_z += dLdz_0
            adj_t[0] = adj_t[0] - dLdt_0
        return adj_z.view(bs, *z_shape), adj_t, adj_p, None

In [11]:
class NeuralODE(nn.Module):
    def __init__(self, func):
        super(NeuralODE, self).__init__()
        assert isinstance(func, ODEF)
        self.func = func

    def forward(self, z0, t=None, return_whole_sequence=False):
        if t is None:
            t = torch.tensor([0., 1.]).to(device)
        t = t.to(z0.device)
        z = ODEAdjoint.apply(z0, t, self.func.flatten_parameters(), self.func)
        if return_whole_sequence:
            return z
        else:
            return z[-1]

In [12]:
# Define the Encoder for the Latent ODE model
class Encoder(nn.Module):
    def __init__(self, input_dim, latent_dim, hidden_dim):
        super(Encoder, self).__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        
        # RNN to process sequential data
        self.gru = nn.GRU(input_dim + 1, hidden_dim, batch_first=True)  # +1 for time delta
        
        # Output mean and log variance for the latent state
        self.fc_mean = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
        
    def forward(self, x, time_delta):
        # Combine position and time data
        combined = torch.cat([x, time_delta.unsqueeze(-1)], dim=-1)
        
        # Process with GRU
        _, h_n = self.gru(combined.unsqueeze(0))
        h_n = h_n.squeeze(0)
        
        # Get latent parameters
        mean = self.fc_mean(h_n)
        logvar = self.fc_logvar(h_n)
        
        return mean, logvar

In [13]:
# Define the ODE function for the latent space dynamics
class LatentODEFunc(ODEF):
    def __init__(self, latent_dim, hidden_dim):
        super(LatentODEFunc, self).__init__()
        self.latent_dim = latent_dim
        
        # Neural network to approximate the derivative
        self.net = nn.Sequential(
            nn.Linear(latent_dim + 1, hidden_dim),  # +1 for time
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, latent_dim)
        )
        
    def forward(self, z, t):
        # Expand t to match z's dimensions
        t_expanded = t.expand(z.shape[0], 1)
        
        # Concatenate z and t
        z_t = torch.cat([z, t_expanded], dim=1)
        
        # Compute derivative
        return self.net(z_t)

In [14]:
# Define the Decoder to map from latent space back to observation space
class Decoder(nn.Module):
    def __init__(self, latent_dim, output_dim, hidden_dim):
        super(Decoder, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
        
    def forward(self, z):
        return self.net(z)

In [15]:
# Complete Latent ODE model
class LatentODE(nn.Module):
    def __init__(self, input_dim, latent_dim, hidden_dim):
        super(LatentODE, self).__init__()
        self.latent_dim = latent_dim
        
        # Encoder, ODE function, and Decoder
        self.encoder = Encoder(input_dim, latent_dim, hidden_dim).to(device)
        self.ode_func = LatentODEFunc(latent_dim, hidden_dim).to(device)
        self.decoder = Decoder(latent_dim, input_dim, hidden_dim).to(device)
        
        # Neural ODE solver
        self.ode_solver = NeuralODE(self.ode_func).to(device)
        
    def encode(self, x, time_delta):
        mean, logvar = self.encoder(x, time_delta)
        
        # Reparameterization trick for sampling
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        z0 = mean + eps * std
        
        return z0, mean, logvar
    
    def forward(self, x, times, time_delta, return_latent=False):
        # Encode to get initial latent state
        z0, mean, logvar = self.encode(x, time_delta)
        
        # Solve ODE to get latent trajectory
        z_trajectory = self.ode_solver(z0.unsqueeze(0), times, return_whole_sequence=True)
        
        # Decode latent trajectory to observations
        x_reconstructed = self.decoder(z_trajectory)
        
        if return_latent:
            return x_reconstructed, z_trajectory, mean, logvar
        else:
            return x_reconstructed, mean, logvar

In [16]:
# Create uniform time grid for reconstruction and evaluation
min_time = tensor_timestamps[0].item()
max_time = tensor_timestamps[-1].item()
num_uniform_points = 300  # Adjust as needed for smoother interpolation
uniform_times = torch.linspace(min_time, max_time, num_uniform_points).to(device)

In [17]:
# Initialize model
input_dim = 3  # x, y, z coordinates
latent_dim = 6  # Size of latent space (adjust as needed)
hidden_dim = 32  # Size of hidden layers (adjust as needed)
model = LatentODE(input_dim, latent_dim, hidden_dim).to(device)

In [18]:
# Training parameters
num_epochs = 1000
batch_size = 32
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, factor=0.5, min_lr=0.0001)

In [19]:
# Loss function for VAE (reconstruction + KL divergence)
def loss_function(x_reconstructed, x_target, mean, logvar):
    # Reconstruction loss (MSE)
    recon_loss = F.mse_loss(x_reconstructed, x_target)
    
    # KL divergence loss
    kl_loss = -0.5 * torch.sum(1 + logvar - mean.pow(2) - logvar.exp())
    
    # Total loss (with weight on KL term)
    return recon_loss + 0.01 * kl_loss, recon_loss, kl_loss

In [20]:
# For tracking loss
train_losses = []


In [21]:
# Create a tensor of all available timestamps
all_times = tensor_timestamps.unsqueeze(1)  # Shape [num_points, 1]

In [22]:
# Training loop
print("Starting training...")
start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass using all data
    x_reconstructed, mean, logvar = model(tensor_positions, all_times, tensor_time_deltas)
    
    # Calculate loss
    loss, recon_loss, kl_loss = loss_function(
        x_reconstructed.squeeze(1),  # Remove batch dimension
        tensor_positions.unsqueeze(0).expand_as(x_reconstructed).squeeze(1),  # Match dimensions
        mean, 
        logvar
    )
    
    # Backpropagation
    loss.backward()
    
    # Update parameters
    optimizer.step()
    
    # Track loss
    train_losses.append(loss.item())
    
    # Print progress
    if (epoch + 1) % 50 == 0:
        end_time = time.time()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.6f}, Recon Loss: {recon_loss.item():.6f}, KL Loss: {kl_loss.item():.6f}, Time: {end_time - start_time:.2f}s")
        start_time = end_time
        
        # Step the scheduler
        scheduler.step(loss)

print("Training finished!")

Starting training...


RuntimeError: Tensors must have same number of dimensions: got 3 and 2

In [ ]:
# Create uniform time grid for better visualization
uniform_times_tensor = uniform_times.unsqueeze(1)

In [ ]:
# Generate reconstructed trajectory with uniform sampling
model.eval()
with torch.no_grad():
    # Get the initial latent state
    z0, _, _ = model.encode(tensor_positions, tensor_time_deltas)
    
    # Solve ODE to get latent trajectory at uniform time steps
    z_trajectory = model.ode_solver(z0.unsqueeze(0), uniform_times_tensor, return_whole_sequence=True)
    
    # Decode latent trajectory to observations
    uniform_reconstructed = model.decoder(z_trajectory)

In [ ]:
# Convert to numpy for plotting
original_positions = positions
uniform_times_np = uniform_times.cpu().numpy()
uniform_reconstructed_np = uniform_reconstructed.squeeze().cpu().numpy()

In [ ]:
# Apply inverse scaling to get back to original scale
uniform_reconstructed_np = scaler.inverse_transform(uniform_reconstructed_np)

In [ ]:
# Extract coordinates for clarity
orig_x, orig_y, orig_z = original_positions[:, 0], original_positions[:, 1], original_positions[:, 2]
recon_x, recon_y, recon_z = uniform_reconstructed_np[:, 0], uniform_reconstructed_np[:, 1], uniform_reconstructed_np[:, 2]

In [ ]:
# Create 3D plot
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# Plot original data points
ax.scatter(orig_x, orig_y, orig_z, c='blue', marker='o', s=10, label='Original Data')

# Plot reconstructed trajectory
ax.plot(recon_x, recon_y, recon_z, c='red', linewidth=2, label='Reconstructed Trajectory')

# Set labels and title
ax.set_xlabel('X Position')
ax.set_ylabel('Y Position')
ax.set_zlabel('Z Position')
ax.set_title('UAV Trajectory: Original vs Latent ODE Reconstruction')
ax.legend()

plt.savefig('uav_trajectory_reconstruction.png', dpi=300, bbox_inches='tight')
plt.close()

In [ ]:
# Create individual plots for each dimension vs time
fig, axs = plt.subplots(3, 1, figsize=(12, 15))

# Original timestamps
orig_times = timestamps - timestamps[0]

# X position
axs[0].scatter(orig_times, orig_x, c='blue', s=10, label='Original')
axs[0].plot(uniform_times_np, recon_x, c='red', linewidth=2, label='Reconstructed')
axs[0].set_ylabel('X Position')
axs[0].legend()
axs[0].set_title('X Position over Time')

# Y position
axs[1].scatter(orig_times, orig_y, c='blue', s=10, label='Original')
axs[1].plot(uniform_times_np, recon_y, c='red', linewidth=2, label='Reconstructed')
axs[1].set_ylabel('Y Position')
axs[1].legend()
axs[1].set_title('Y Position over Time')

# Z position
axs[2].scatter(orig_times, orig_z, c='blue', s=10, label='Original')
axs[2].plot(uniform_times_np, recon_z, c='red', linewidth=2, label='Reconstructed')
axs[2].set_xlabel('Time (s)')
axs[2].set_ylabel('Z Position')
axs[2].legend()
axs[2].set_title('Z Position over Time')

plt.tight_layout()
plt.savefig('uav_dimensions_over_time.png', dpi=300, bbox_inches='tight')
plt.show()